# Modelling NIQS Hardware

In [ ]:
import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
from qiskit.quantum_info import DensityMatrix
from scipy.linalg import sqrtm

np.set_printoptions(precision=2)

In [82]:
def partial_trace(X, discard_first = True):
    d = X.shape[0]
    d_red = int(np.sqrt(d))
    Y = np.zeros((d_red, d_red), dtype = "complex128")
    I = np.eye(d_red)
    
    for i in range(d_red):
        basis_vec = np.zeros((d_red, 1),  dtype = "complex128")
        basis_vec[i, 0] = 1
        
        if discard_first:
            basis_vec = np.kron(basis_vec, I)
        else:
            basis_vec = np.kron(I, basis_vec)
        
        Y = Y + basis_vec.T@X@basis_vec
    
    return Y


def state_fidelity(A, B):
    sqrtA = sqrtm(A)
    fidelity = np.trace(sqrtm(sqrtA@B@sqrtA))
    return np.abs(fidelity)


def apply_map(state, choi):
    d = state.shape[0]
    
    #reshuffle
    choi = choi.reshape(d,d,d,d).swapaxes(1,2).reshape(d**2, d**2)
    
    #flatten
    state = state.reshape(-1, 1)
    
    state = (choi@state).reshape(d, d)
    return state
    

def prepare_input(config):
    """1 = |0>, 2 = |1>, 3 = |+>, 4 = |->, 5 = |+i>, 6 = |-i>"""
    n = len(config)
    circuit = qk.QuantumCircuit(n)
    for i, gate in enumerate(config):
        if gate == 2:
            circuit.x(i)
        if gate == 3:
            circuit.h(i)
        if gate == 4:
            circuit.x(i)
            circuit.h(i)
        if gate == 5:
            circuit.h(i)
            circuit.s(i)
        if gate == 6:
            circuit.x(i)
            circuit.h(i)
            circuit.s(i)
        
            
    rho = DensityMatrix(circuit)
    return rho.data


def generate_ginibre(dim1, dim2, real = False):
    ginibre = np.random.normal(0, 1, (dim1, dim2))
    if not real:
         ginibre = ginibre + 1j*np.random.normal(0, 1, (dim1, dim2))
    return ginibre

def generate_state(dim1, dim2):
    X = generate_ginibre(dim1, dim2)
    
    state = X@X.conj().T/np.trace(X@X.conj().T)
    return state

def generate_choi(X):
    d = int(np.sqrt(X.shape[0]))  # dim of Hilbert space
    I = np.eye(d)

    #partial trace
    Y = partial_trace(X@(X.conj().T), discard_first = True)
    sqrtYinv = np.linalg.inv(sqrtm(Y))

    #choi
    choi = np.kron(I, sqrtYinv)@X@(X.conj().T)@np.kron(I, sqrtYinv)
    
    return choi


class Adam():
    def __init__(self, dims, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        
        self.t = 0
        self.m = np.zeros(dims, dtype="complex128")
        self.v = np.zeros(dims, dtype="complex128")


    def __call__(self, gradient):
        self.t += 1

        self.m = self.beta1 * self.m + (1 - self.beta1) * gradient
        self.v = self.beta2 * self.v + (1 - self.beta2) * np.abs(gradient)**2

        m_hat = self.m / (1 - self.beta1**self.t)
        v_hat = self.v / (1 - self.beta2**self.t)
        gradient_modified = m_hat / (np.sqrt(v_hat) + self.eps)

        return gradient_modified
    
    
class ModelQuantumMap:
    def __init__(self, n, rank, lr, h):
        self.n = n
        self.rank = rank
        self.lr = lr
        self.h = h
        
        self.d = 2**n
        self.X_model = generate_ginibre(self.d**2, self.rank)
        
        self.adam = Adam(dims = (self.d**2, self.rank))
        self.fid_list = []  
        
    def train(self, choi_target, num_iter, use_adam=False):
        for step in range(num_iter):  
            grad_matrix = np.zeros((dim1, dim2), dtype="complex128")
            indicies = [np.random.randint(1,6) for i in range(n)]
            state_input = prepare_input(indicies)
            state_target = apply_map(state_input, choi_target)
            h = self.h
    
            for i in range(self.d**2):
                for j in range(self.rank):
                    
                    #Finite difference, real value
                    self.X_model[i, j] += h
                    choi_plus = generate_choi(self.X_model)
                    state_plus = apply_map(state_input, choi_plus)
                    fid_plus = state_fidelity(state_plus, state_target)

                    self.X_model[i, j] -= 2*h
                    choi_minus = generate_choi(self.X_model)
                    state_minus = apply_map(state_input, choi_minus)
                    fid_minus = state_fidelity(state_minus, state_target)
                    self.X_model[i, j] += h

                    grad = (fid_plus-fid_minus)/h
                    grad_matrix[i, j] += grad 

                    #Finite difference, imaginary value
                    self.X_model[i, j] += 1j*h
                    choi_plus = generate_choi(self.X_model)
                    state_plus = apply_map(state_input, choi_plus)
                    fid_plus = state_fidelity(state_plus, state_target)

                    self.X_model[i, j] -= 2j*h
                    choi_minus = generate_choi(self.X_model)
                    state_minus = apply_map(state_input, choi_minus)
                    fid_minus = state_fidelity(state_minus, state_target)
                    self.X_model[i, j] += 1j*h

                    grad = 1j*(fid_plus-fid_minus)/h
                    grad_matrix[i, j] += grad

            if use_adam:
                grad_matrix = self.adam(grad_matrix)
                
            self.X_model += self.lr*grad_matrix

            choi_model = generate_choi(self.X_model)
            state_model = apply_map(state_input, choi_model)
            fid = state_fidelity(state_model, state_target)
            
            self.fid_list.append(fid)
            print(f"{step}: {fid:.3f}")

In [ ]:
n = 3
d = 2**n

np.random.seed(42)

X_target = generate_ginibre(d**2, 2)
choi_target = generate_choi(X_target)

model1 = ModelQuantumMap(n = 3, 
                         rank = 2, 
                         lr = 0.5, 
                         h = 1e-4)

model1.train(choi_target = choi_target, 
             num_iter = 1000, 
             use_adam = False)

0: 0.351
1: 0.386
2: 0.265
3: 0.482
4: 0.315
5: 0.408
6: 0.419
7: 0.390
8: 0.418
9: 0.483
10: 0.290
11: 0.453
12: 0.626
13: 0.397
14: 0.627
15: 0.477
16: 0.388
17: 0.646
18: 0.333
19: 0.551
20: 0.423
21: 0.474
22: 0.412
23: 0.572
24: 0.552
25: 0.447
26: 0.433
27: 0.676
28: 0.440
29: 0.529
30: 0.480


In [ ]:
n = 3
d = 2**n

np.random.seed(42)

X_target = generate_ginibre(d**2, 2)
choi_target = generate_choi(X_target)

model2 = ModelQuantumMap(n = 3, 
                         rank = 2, 
                         lr = 0.05, 
                         h = 1e-4)

model2.train(choi_target = choi_target, 
            num_iter = 1000, 
            use_adam = True)